In [1]:
import pandas as pd
import os

# File Paths
STUDENT_FILE = "Course List 217.xlsx"
SEATS_LA_LC_LH = "LA_LC_LH_final.xlsx"
SEATS_CC_KR = "CC_KR_final.xlsx"
OUTPUT_FILE = "allocated_seats.xlsx"

# Read Student Data
if not os.path.exists(STUDENT_FILE):
    print(f"⚠️ Error: {STUDENT_FILE} not found.")
    exit()

df_students = pd.read_excel(STUDENT_FILE)

# Validate required columns
required_columns = ["Roll No", "Name"]
if not all(col in df_students.columns for col in required_columns):
    print(f"⚠️ Error: The file must contain these columns: {required_columns}")
    exit()

# Read Seat Data
if not os.path.exists(SEATS_LA_LC_LH) or not os.path.exists(SEATS_CC_KR):
    print(f"⚠️ Error: Seat allocation files not found.")
    exit()

df_seats_la_lc_lh = pd.read_excel(SEATS_LA_LC_LH)
df_seats_cc_kr = pd.read_excel(SEATS_CC_KR)

# User Input for Room Constraints
room_constraints = {}

num_rooms = int(input("🔹 How many rooms do you want to specify? "))

for _ in range(num_rooms):
    room = input("\nEnter Room Number (e.g., LA 001, CC 101): ").strip()

    if room.startswith(("LA", "LC", "LH")):
        # LA/LC/LH rooms need color constraints for Left, Middle, Right
        left_colors = input(f"Enter allowed colors for {room} (Left), comma-separated: ").strip().split(",")
        middle_colors = input(f"Enter allowed colors for {room} (Middle), comma-separated: ").strip().split(",")
        right_colors = input(f"Enter allowed colors for {room} (Right), comma-separated: ").strip().split(",")

        room_constraints[room] = {
            "Left": [c.strip() for c in left_colors if c.strip()],
            "Middle": [c.strip() for c in middle_colors if c.strip()],
            "Right": [c.strip() for c in right_colors if c.strip()]
        }

    elif room.startswith(("CC", "KR")):
        # CC/KR rooms need even/odd constraints
        parity = input(f"Should {room} have 'Even' or 'Odd' seat numbers? ").strip()
        if parity.lower() not in ["even", "odd"]:
            print("⚠️ Invalid input! Defaulting to 'Even'.")
            parity = "Even"

        room_constraints[room] = {"Parity": [parity.capitalize()]}

# Allocate Seats
allocated_seats = []
remaining_students = df_students.copy()

for room, constraints in room_constraints.items():
    if room.startswith(("LA", "LC", "LH")):
        # Assign seats based on Left, Middle, Right constraints
        for position, colors in constraints.items():
            valid_seats = df_seats_la_lc_lh[(df_seats_la_lc_lh["Room Number"] == room) &
                                            (df_seats_la_lc_lh["Position"] == position) &
                                            (df_seats_la_lc_lh["Color"].isin(colors))]

            if valid_seats.empty:
                print(f"⚠️ No valid seats for {room} - {position} with colors {colors}. Skipping...")
                continue

            num_seats = min(len(valid_seats), len(remaining_students))
            assigned_students = remaining_students.iloc[:num_seats].copy()
            assigned_students["Seat Number"] = valid_seats.iloc[:num_seats]["Seat Number"].values
            assigned_students["Room"] = room
            assigned_students["Signature"] = ""

            allocated_seats.append(assigned_students)
            remaining_students = remaining_students.iloc[num_seats:]

    elif room.startswith(("CC", "KR")):
        # Assign seats based on Even/Odd constraint
        valid_seats = df_seats_cc_kr[(df_seats_cc_kr["Room Number"] == room) &
                                     (df_seats_cc_kr["Parity"].isin(constraints.get("Parity", [])))]

        if valid_seats.empty:
            print(f"⚠️ No valid seats for {room} with parity {constraints.get('Parity', [])}. Skipping...")
            continue

        num_seats = min(len(valid_seats), len(remaining_students))
        assigned_students = remaining_students.iloc[:num_seats].copy()
        assigned_students["Seat Number"] = valid_seats.iloc[:num_seats]["Seat Number"].values
        assigned_students["Room"] = room
        assigned_students["Signature"] = ""

        allocated_seats.append(assigned_students)
        remaining_students = remaining_students.iloc[num_seats:]

# Save Output
df_final = pd.concat(allocated_seats, ignore_index=True)[["Roll No", "Name", "Seat Number", "Room", "Signature"]]
df_final.to_excel(OUTPUT_FILE, index=False)

print(f"\n✅ Seat allocation completed! The file is saved as: {OUTPUT_FILE}")


🔹 How many rooms do you want to specify?  2

Enter Room Number (e.g., LA 001, CC 101):  LA 001
Enter allowed colors for LA 001 (Left), comma-separated:  Yellow
Enter allowed colors for LA 001 (Middle), comma-separated:  Yellow
Enter allowed colors for LA 001 (Right), comma-separated:  Yellow

Enter Room Number (e.g., LA 001, CC 101):  CC 101
Should CC 101 have 'Even' or 'Odd' seat numbers?  Even



✅ Seat allocation completed! The file is saved as: allocated_seats.xlsx
